# ყველა ფაილის მისამართის ამოღება

In [67]:
!pip install librosa
!pip install pydub

In [1]:
import glob
import os
from pydub import AudioSegment

# audio ფაილების გადაყვანა wav-ში

In [2]:
def convert_to_wav(files):
    for src in files:
        dst = src[:-3] + "wav"
        s = AudioSegment.from_file(src)
        s.export(dst, format="wav")

In [3]:
folders = os.listdir('data')
for sub_dir in folders:
#     print(sub_dir)
    mp3_files = glob.glob(os.getcwd() + '/data/' + '2' + '/*.mp3')
    convert_to_wav(mp3_files)
#     m4a_files = glob.glob(os.getcwd() + '/data/' + '2' + '/*.m4a')
#     mp4_files = glob.glob(os.getcwd() + '/data/' + '2' + '/*.mp4')
#     aac_files = glob.glob(os.getcwd() + '/data/' + '2' + '/*.aac')
#     files = mp3_files + m4a_files + mp4_files + aac_files
#     რატომღაც ამას არ შვება :(
#     convert_to_wav(files) 

In [4]:
import pandas as pd
import librosa
import librosa.display

##  <font color =green>extract_features ვარგა. ქვემოთ რაც წერია ყველაფერს სწორად შვება</font>

In [5]:
import numpy as np

def get_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = get_mfcc(audio, sample_rate)
        zero_crossing_rate = get_zero_crossing_rate(audio)
#         other features
        return [mfccs, zero_crossing_rate]
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None 

In [6]:
def get_all_features():
    folders = os.listdir('data')
    folders.sort()
    all_features = []
    for label, sub_dir in enumerate(folders): #label {0; 4}
        for file_name in glob.glob(os.getcwd() + '/data/' + sub_dir + '/*.wav'):
#             print("Extracting file ", file_name)
            try:
                features = get_features(file_name)
            except Exception as e:
                print("Extraction error")
                continue
            all_features.append(features +  [label + 1])
    data = pd.DataFrame(all_features, columns=['mfccs', 'zero_crossing_rate' , 'class_label'])
    return data

In [7]:
def get_mfcc(audio, sr):
    #Mel Frequency Cepstral Coefficient
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    mfccsscaled = np.mean(mfccs.T,axis=0)
#     mfccs - (40, 61), mfccsscaled - (40,)
    return mfccsscaled

In [8]:
def get_zero_crossing_rate(audio):
    #ZERO_CROSSING_RATE
    rate = librosa.feature.zero_crossing_rate(audio)
#     rate - (1, 61)
    return rate

In [16]:
def get_spectral_centroid(audio):
    #SPECTRAL_CENTROID
    sp = librosa.feature.spectral_centroid(audio)
#     sp - (1, 61)
    return sp

In [17]:
def get_spectral_roll_ff(audio, sr):
    # Spectral-roll off
    S, phase = librosa.magphase(librosa.stft(audio))
    a = librosa.feature.spectral_rolloff(S=S, sr=sr)
#     a - (1, 61)
    return a

In [18]:
def get_spectral_flux(audio, sr):
    # Spectral flux
    onset_env = librosa.onset.onset_strength(y=audio, sr=sr)
#     onset_env - (61,)
    return onset_env

In [19]:
def get_chroma_features(audio, sr):    
    # Chroma features (needs scaling)
    chroma_stft = librosa.feature.chroma_stft(y=audio, sr=sr) #N1
    chroma_cens = librosa.feature.chroma_cens(y=audio, sr=sr) #Chroma energy normalized statistics
    chroma_cq = librosa.feature.chroma_cqt(y=audio, sr=sr)
#     samive - (12, 61)
    return [chroma_stft, chroma_cens, chroma_cq]

In [20]:
def get_pitch(audio, sr):
    # Pitch (needs scaling)
    pitches, magnitudes = librosa.piptrack(y=audio, sr=sr)
#     pitches - (1025, 61)
    return pitches

In [13]:
data = get_all_features()

In [55]:
labels = [[i] for i in data['class_label']]
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(labels)
print(y_onehot.shape)

(166, 5)


/home/tamar/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
